<a href="https://colab.research.google.com/github/snipaid-nlg/models/blob/main/news-headline-generation/news-headline-generation-with-mGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# News Headline Generation with mGPT
Evaluating mGPT on the task of generating headlines from journalistic text in german.

## Installs and Imports

In [ ]:
!pip install -q transformers datasets accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 KB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset, Dataset
from transformers.pipelines.pt_utils import KeyDataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer, pipeline
import pandas as pd
from tqdm.auto import tqdm

## Loading and transforming 10kGNAD
We load the [Ten Thousand German News Articles Dataset](https://tblock.github.io/10kGNAD/) from Huggingface. The first sentence of a text (up to the first period) is a headline, so we split the text and use the second part. A Suffix `Headline:` is added to prompt the model.

In [ ]:
gnad10 = load_dataset("gnad10")
df = pd.DataFrame(gnad10["train"])
df[['headline', 'fulltext']] = df["text"].str.split('.', n=1, expand=True)
df.drop(['label','text'], axis=1, inplace=True)
df['fulltext'] = df['fulltext'] + " Titel:"
df.head()

Generating train split:   0%|          | 0/9245 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1028 [00:00<?, ? examples/s]

Dataset gnad10 downloaded and prepared to /root/.cache/huggingface/datasets/gnad10/default/1.1.0/3a8445be65795ad88270af4d797034c3d99f70f8352ca658c586faf1cf960881. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

,headline,fulltext
0,21-Jähriger fällt wohl bis Saisonende aus,Wien – Rapid muss wohl bis Saisonende auf Off...
1,"Erfundene Bilder zu Filmen, die als verloren g...",Wien – Die Filmgeschichte ist ein Friedhof de...
2,Der frischgekürte CEO Sundar Pichai setzt auf ...,Die Atmosphäre im Silicon Valley ist rau. Da ...
3,"Putin: ""Einigung, dass wir Menge auf Niveau vo...",Moskau – Die russischen Ölproduzenten wollen ...
4,Estland sieht den künftigen österreichischen P...,Wien/Tallinn/Pskow – Die Eltern des künftigen...


Converting the pandas dataframe to a Huggingface dataset object

In [ ]:
dataset = Dataset.from_pandas(df)

## Loading the model

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/mGPT")
model = GPT2LMHeadModel.from_pretrained("sberbank-ai/mGPT", device_map="auto")

In [ ]:
template = {
      "min_new_tokens": 5,
      "max_new_tokens": 30,
      "length_penalty": 1.0,
      "no_repeat_ngram_size": 4,
      "num_beams": 4,
      #"repetition_penalty": 1.0,
      #"diversity_penalty": 0.0,
      "num_beam_groups": 1,
      "do_sample": False,
      "top_k": 0,
      "top_p": 0.8,
      "temperature": 1.0,
      #"early_stopping": True,
      "pad_token_id": 1,
      "eos_token_id": 5,
      "num_return_sequences": 1,
      "return_full_text": False,
}

In [ ]:
pipe = pipeline("text-generation",
                device=0,
                model=model, 
                tokenizer=tokenizer,
                **template)

In [ ]:
for out in tqdm(pipe(KeyDataset(dataset, "fulltext"))):
    print(out)

  0%|          | 0/9245 [00:00<?, ?it/s]

[{'generated_text': ' Rapid gewinnt mit 2:0 gegen Admira.'}]
[{'generated_text': ' Der Film, der Film, die Film, die Verfilm, der Verfilm der Verfilm, Verfilm, die Verke, die Verf'}]
[{'generated_text': '\nGoogle-Chef Pichai: Google-Mitarbeiter sind Choleriker\nGoogle-Gründer Larry Page: Google-Gründer Pichai'}]
[{'generated_text': ' Ölpreise: Putin will Ölförderung einfrieren'}]
[{'generated_text': ' der Partei der Partei, der Partei. der Partei, die Partei der Partei in der Partei, in der Partei. Partei der Partei der Regierung der Partei'}]
[{'generated_text': ' TRODAT TROTEC HOLDING GMBH, WELS\nPermalink: https://www.f'}]
[{'generated_text': '\nHeracles Almelo – FC Utrecht 2:0 (1:0)\nHinspiel: 1:1 (0:1)\n'}]
[{'generated_text': ' "Guten Morgen Österreich"'}]


KeyboardInterrupt: ignored